For simplicity and education:
    functionality has been removed
        submission path
        versioning

In [32]:
import pandas as pd
import numpy as np

In [33]:
import os
dirname = os.getcwd()
print(dirname)

/home/jupyterlab/pipeline-202102


In [34]:
os.environ['MPLCONFIGDIR'] = dirname
import matplotlib

In [35]:
import xlsxwriter
import xlrd

In [36]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-02-09T08:56:37.990407


In [37]:
print(my_date.strftime('%Y-%m-%dT%H:%M:%S.%f%z'))

2021-02-09T08:56:37.990407


In [38]:
runid = "R" + my_date.strftime('%Y%m%d%H%M%S%f%z')
runid

'R20210209085637990407'

In [39]:
filename = dirname + '/Data.xlsx'
filedf = pd.read_excel(filename,engine='openpyxl', sheet_name=None, header=None)

In [40]:
sheetnames = list(filedf.keys())

In [41]:
def validate_date(date_text):
    try:
        format_string = '%Y-%m-%d %H:%M:%S'
        datetime.strptime(date_text, format_string)
        return "OK"
    except ValueError:
        return None

def GetDataType(row_in):
    valueasstring = str(row_in['CellValue'])
    if validate_date(valueasstring) != None:
        return 'Date'
    elif valueasstring.isnumeric():
        return 'Number'
    else:
        return 'Text'

def add1_2row(row_in):
    return row_in['RowID'] + 1


def add1_2column(row_in):
    return row_in['ColumnID'] + 1

In [42]:
celldbstagedf = pd.DataFrame()
sheetdf = filedf['DataMap'].copy()
sheetdf["RowID"] = sheetdf.index
#sheetdf

In [43]:
for currentsheetname in sheetnames:

    sheetdf = filedf[currentsheetname].copy()
    sheetdf["RowID"] = sheetdf.index
    celldb = pd.melt(sheetdf, id_vars=['RowID'])
    celldb = celldb.rename(columns={"variable": "ColumnID", "value": "CellValue"})
    celldb["CellType"] = "Unset"
    celldb["FileName"] = runid
    celldb["SheetName"] = currentsheetname
    
    celldb["ColumnID"] = celldb["ColumnID"]

    try:
        celldb["CellType"] = celldb.apply(GetDataType, axis=1)
        celldb["RowID"] = celldb.apply(add1_2row, axis=1)
        celldb["ColumnID"] = celldb.apply(add1_2column, axis=1)

    except:

        celldb["CellType"] = 'unknown'
        celldb["RowID"] = 0
        celldb["ColumnID"] = 0

    celldbstagedf = celldbstagedf.append(celldb, ignore_index=True)

    celldb = pd.DataFrame()



In [44]:
fullcelldb = celldbstagedf.copy()

In [45]:
def GetTableFromCells(Dataset_in):
    Cells = pd.DataFrame()
    Cells = fullcelldb[(fullcelldb['SheetName'] == Dataset_in)].copy()
    Table = pd.DataFrame()
    Table = Cells.pivot_table(index = ['RowID'], values = 'CellValue',columns = 'ColumnID', aggfunc = 'max')
    Headings = Table.iloc[0]
    HeadingsList = list(Headings)
    HeadingsListTuple = enumerate(HeadingsList,start=1)
    HeadingsListDict = dict(HeadingsListTuple)
    Table = Table.rename(columns=HeadingsListDict)
    Table = Table.drop([1])
    Columns = Table.columns.tolist()
    return Table, Columns

In [46]:
Dataset_Table, Dataset_Columns = GetTableFromCells('DataSet')

In [47]:
Dataset_Table

ColumnID,DataSet,SheetName,ReferenceTextStart,ReferenceTextEnd,RowFromReferenceText,ColumnFromReferenceText,FieldPosition,FieldID,FieldDescription,TableOrientation
RowID,,,,,,,,,,
2,Ages,AgeGenderSheet,Age Table,Gender Table,2,1,1,Name,Name,Down
3,Ages,AgeGenderSheet,Age Table,Gender Table,2,2,2,Age,Age,Down
4,Genders,AgeGenderSheet,Gender Table,DOB Table,2,0,1,Name,Name,Down
5,Genders,AgeGenderSheet,Gender Table,DOB Table,2,1,2,Gender,Gender,Down
6,DoB,AgeGenderSheet,DOB Table,NaN,2,0,1,Name,Name,Down
7,DoB,AgeGenderSheet,DOB Table,NaN,2,1,2,DoB,D.O.B,Down
8,MaritalStatus,MaritalDepartmentSheet,Marital Status Table,NaN,2,0,1,Name,Name,NaN
9,MaritalStatus,MaritalDepartmentSheet,Marital Status Table,NaN,2,1,2,Status,Status,NaN
10,Department,MaritalDepartmentSheet,Department Table,NaN,2,0,1,Name,Name,NaN


In [77]:
ReferenceText_Table = Dataset_Table[['DataSet','SheetName','ReferenceTextStart','ReferenceTextEnd']]
ReferenceText_Table = ReferenceText_Table.drop_duplicates()
ReferenceText_Table

ColumnID,DataSet,SheetName,ReferenceTextStart,ReferenceTextEnd
RowID,,,,
2,Ages,AgeGenderSheet,Age Table,Gender Table
4,Genders,AgeGenderSheet,Gender Table,DOB Table
6,DoB,AgeGenderSheet,DOB Table,NaN
8,MaritalStatus,MaritalDepartmentSheet,Marital Status Table,NaN
10,Department,MaritalDepartmentSheet,Department Table,NaN
12,Jobs,JobSheet,Job Description Table,NaN
15,HomeLocation,HomeSheet,Home Location Table,NaN


In [49]:
fullcelldb.head()

,RowID,ColumnID,CellValue,CellType,FileName,SheetName
0,1,1,NaN,Text,R20210209085637990407,AgeGenderSheet
1,2,1,NaN,Text,R20210209085637990407,AgeGenderSheet
2,3,1,NaN,Text,R20210209085637990407,AgeGenderSheet
3,4,1,NaN,Text,R20210209085637990407,AgeGenderSheet
4,5,1,NaN,Text,R20210209085637990407,AgeGenderSheet


In [50]:
## df1.merge(df2, left_on='lkey', right_on='rkey')
## new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])

Anchor_Table = fullcelldb.merge(ReferenceText_Table,how='right', left_on=['SheetName','CellValue'], right_on=['SheetName','ReferenceTextStart'])

In [61]:
Anchor_Table = Anchor_Table.rename(columns={"RowID": "AnchorRowID", "ColumnID": "AnchorColumnID"})
Anchor_Table = Anchor_Table.drop(['CellValue', 'CellType'], axis=1)
Anchor_Table

,AnchorRowID,AnchorColumnID,FileName,SheetName,DataSet,ReferenceTextStart
0,5,2,R20210209085637990407,AgeGenderSheet,Ages,Age Table
1,14,4,R20210209085637990407,AgeGenderSheet,Genders,Gender Table
2,23,1,R20210209085637990407,AgeGenderSheet,DoB,DOB Table
3,3,1,R20210209085637990407,MaritalDepartmentSheet,MaritalStatus,Marital Status Table
4,2,5,R20210209085637990407,MaritalDepartmentSheet,Department,Department Table
5,6,3,R20210209085637990407,JobSheet,Jobs,Job Description Table
6,8,1,R20210209085637990407,HomeSheet,HomeLocation,Home Location Table


In [84]:
End_Table = fullcelldb.merge(ReferenceText_Table,how='inner', left_on=['SheetName','CellValue'], right_on=['SheetName','ReferenceTextEnd'])
End_Table.dropna(subset=['ReferenceTextEnd'],inplace=True)
End_Table = End_Table.rename(columns={"RowID": "EndRowID", "ColumnID": "EndColumnID"})
End_Table = End_Table.drop(['CellValue', 'CellType', 'FileName', 'ReferenceTextStart'], axis=1)
#End_Table[End_Table['DataSet']=='Ages']
End_Table

,EndRowID,EndColumnID,SheetName,DataSet,ReferenceTextEnd
108,23,1,AgeGenderSheet,Genders,DOB Table
109,14,4,AgeGenderSheet,Ages,Gender Table


In [85]:
Datamap_Table, Datamap_Columns = GetTableFromCells('DataMap')

In [86]:
Datamap_Table

ColumnID,SheetName,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,DataCategory,DataScope,Column Header,DataSet
RowID,,,,,,,,,,
2,AgeGenderSheet,Age Table,2,1,Name,Ages,NaN,NaN,Name,Ages
3,AgeGenderSheet,Age Table,2,2,Age,Ages,NaN,NaN,Age,Ages
4,AgeGenderSheet,Gender Table,2,0,Name,Genders,NaN,NaN,Name,Genders
5,AgeGenderSheet,Gender Table,2,1,Gender,Genders,NaN,NaN,Gender,Genders
6,AgeGenderSheet,DOB Table,2,0,Name,DoB,NaN,NaN,Name,DoB
7,AgeGenderSheet,DOB Table,2,1,D.O.B,DoB,NaN,NaN,D.O.B,DoB
8,MaritalDepartmentSheet,Marital Status Table,2,0,Name,Marital Status,NaN,NaN,Name,MaritalStatus
9,MaritalDepartmentSheet,Marital Status Table,2,1,Status,Marital Status,NaN,NaN,Status,MaritalStatus
10,MaritalDepartmentSheet,Department Table,2,0,Name,Department,NaN,NaN,Name,Department


In [87]:
MetaData_Table, MetaData_Columns = GetTableFromCells('MetaData')

In [88]:
MetaData_Table

ColumnID,Database,DataSetID,Schema,TableName
RowID,,,,
2,data,Ages,raw,Ages
3,data,Genders,raw,Genders
4,data,DoB,raw,DoB
5,data,MaritalStatus,raw,MaritalStatus
6,data,Department,raw,Department
7,data,Jobs,raw,Jobs
8,data,HomeLocation,raw,HomeLocation


In [89]:
FullDataMap_Table = Anchor_Table.merge(
        Datamap_Table,how='right',
        left_on=['DataSet','SheetName'],
        right_on=['DataSet','SheetName'])

FullDataMap_Table = FullDataMap_Table.drop(['FileName'], axis=1)

FullDataMap_Table = FullDataMap_Table.merge(
        End_Table,how='left',
        left_on=['DataSet','SheetName'],
        right_on=['DataSet','SheetName'])

FullDataMap_Table

,AnchorRowID,AnchorColumnID,SheetName,DataSet,ReferenceTextStart,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,DataCategory,DataScope,Column Header,EndRowID,EndColumnID,ReferenceTextEnd
0,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,NaN,NaN,Name,14.0,4.0,Gender Table
1,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,NaN,NaN,Age,14.0,4.0,Gender Table
2,14,4,AgeGenderSheet,Genders,Gender Table,Gender Table,2,0,Name,Genders,NaN,NaN,Name,23.0,1.0,DOB Table
3,14,4,AgeGenderSheet,Genders,Gender Table,Gender Table,2,1,Gender,Genders,NaN,NaN,Gender,23.0,1.0,DOB Table
4,23,1,AgeGenderSheet,DoB,DOB Table,DOB Table,2,0,Name,DoB,NaN,NaN,Name,NaN,NaN,NaN
5,23,1,AgeGenderSheet,DoB,DOB Table,DOB Table,2,1,D.O.B,DoB,NaN,NaN,D.O.B,NaN,NaN,NaN
6,3,1,MaritalDepartmentSheet,MaritalStatus,Marital Status Table,Marital Status Table,2,0,Name,Marital Status,NaN,NaN,Name,NaN,NaN,NaN
7,3,1,MaritalDepartmentSheet,MaritalStatus,Marital Status Table,Marital Status Table,2,1,Status,Marital Status,NaN,NaN,Status,NaN,NaN,NaN
8,2,5,MaritalDepartmentSheet,Department,Department Table,Department Table,2,0,Name,Department,NaN,NaN,Name,NaN,NaN,NaN
9,2,5,MaritalDepartmentSheet,Department,Department Table,Department Table,2,1,Department,Department,NaN,NaN,Department,NaN,NaN,NaN


In [90]:
FullDataPublish_Table = FullDataMap_Table.merge(
        fullcelldb,how='left',
        left_on=['SheetName'],
        right_on=['SheetName'])
FullDataPublish_Table

,AnchorRowID,AnchorColumnID,SheetName,DataSet,ReferenceTextStart,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,...,DataScope,Column Header,EndRowID,EndColumnID,ReferenceTextEnd,RowID,ColumnID,CellValue,CellType,FileName
0,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,NaN,Name,14.0,4.0,Gender Table,1,1,NaN,Text,R20210209085637990407
1,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,NaN,Name,14.0,4.0,Gender Table,2,1,NaN,Text,R20210209085637990407
2,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,NaN,Name,14.0,4.0,Gender Table,3,1,NaN,Text,R20210209085637990407
3,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,NaN,Name,14.0,4.0,Gender Table,4,1,NaN,Text,R20210209085637990407
4,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,NaN,Name,14.0,4.0,Gender Table,5,1,NaN,Text,R20210209085637990407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,Location,NaN,NaN,NaN,8,2,NaN,Text,R20210209085637990407
1172,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,Location,NaN,NaN,NaN,9,2,Location,Text,R20210209085637990407
1173,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,Location,NaN,NaN,NaN,10,2,London,Text,R20210209085637990407
1174,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,Location,NaN,NaN,NaN,11,2,Liverpool,Text,R20210209085637990407


In [92]:
def KeepColumn(ColumnID,ColumnFromReferenceText,AnchorColumnID):
    if ColumnID == ColumnFromReferenceText + AnchorColumnID:
        return 1
    return 0

def KeepRow(RowID,RowFromReferenceText,AnchorRowID, EndRowID):
    if RowID > RowFromReferenceText + AnchorRowID and RowID < EndRowID:
        return 1
    return 0
#dc.ColumnID = ds.ColumnFromReferenceText + @AnchorColumn

FullDataPublish_Table['KeepColumn'] = FullDataPublish_Table.apply(lambda x: KeepColumn(x['ColumnID'],x['ColumnFromReferenceText'],x['AnchorColumnID']), axis=1)
FullDataPublish_Table['KeepRow'] = FullDataPublish_Table.apply(lambda x: KeepRow(x['RowID'],x['RowFromReferenceText'],x['AnchorRowID'],x['EndRowID']), axis=1)

##df.apply(lambda x: fxy(x['A'], x['B']), axis=1)


#dc.RowID > ds.RowFromReferenceText + @AnchorRow;
FullDataPublish_Table

,AnchorRowID,AnchorColumnID,SheetName,DataSet,ReferenceTextStart,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,...,EndRowID,EndColumnID,ReferenceTextEnd,RowID,ColumnID,CellValue,CellType,FileName,KeepColumn,KeepRow
0,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,1,1,NaN,Text,R20210209085637990407,0,0
1,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,2,1,NaN,Text,R20210209085637990407,0,0
2,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,3,1,NaN,Text,R20210209085637990407,0,0
3,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,4,1,NaN,Text,R20210209085637990407,0,0
4,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,5,1,NaN,Text,R20210209085637990407,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,NaN,NaN,8,2,NaN,Text,R20210209085637990407,1,0
1172,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,NaN,NaN,9,2,Location,Text,R20210209085637990407,1,0
1173,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,NaN,NaN,10,2,London,Text,R20210209085637990407,1,0
1174,8,1,HomeSheet,HomeLocation,Home Location Table,Home Location Table,2,1,Location,Home Location,...,NaN,NaN,NaN,11,2,Liverpool,Text,R20210209085637990407,1,0


In [93]:
Data2BPublished_Cells = FullDataPublish_Table[(FullDataPublish_Table['KeepRow']==1) & (FullDataPublish_Table['KeepColumn']==1)]
Data2BPublished_Cells

,AnchorRowID,AnchorColumnID,SheetName,DataSet,ReferenceTextStart,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,...,EndRowID,EndColumnID,ReferenceTextEnd,RowID,ColumnID,CellValue,CellType,FileName,KeepColumn,KeepRow
61,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,8,3,Ringo,Text,R20210209085637990407,1,1
62,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,9,3,Brian,Text,R20210209085637990407,1,1
63,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,10,3,NaN,Text,R20210209085637990407,1,1
64,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,11,3,NaN,Text,R20210209085637990407,1,1
65,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,12,3,NaN,Text,R20210209085637990407,1,1
66,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,13,3,NaN,Text,R20210209085637990407,1,1
223,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,8,4,22,Number,R20210209085637990407,1,1
224,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,9,4,89,Number,R20210209085637990407,1,1
225,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,10,4,NaN,Text,R20210209085637990407,1,1
226,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,11,4,NaN,Text,R20210209085637990407,1,1


In [94]:
AgesCells = Data2BPublished_Cells[Data2BPublished_Cells['DataSet']=='Ages']
AgesCells

,AnchorRowID,AnchorColumnID,SheetName,DataSet,ReferenceTextStart,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,...,EndRowID,EndColumnID,ReferenceTextEnd,RowID,ColumnID,CellValue,CellType,FileName,KeepColumn,KeepRow
61,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,8,3,Ringo,Text,R20210209085637990407,1,1
62,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,9,3,Brian,Text,R20210209085637990407,1,1
63,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,10,3,NaN,Text,R20210209085637990407,1,1
64,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,11,3,NaN,Text,R20210209085637990407,1,1
65,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,12,3,NaN,Text,R20210209085637990407,1,1
66,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,1,Name,Ages,...,14.0,4.0,Gender Table,13,3,NaN,Text,R20210209085637990407,1,1
223,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,8,4,22,Number,R20210209085637990407,1,1
224,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,9,4,89,Number,R20210209085637990407,1,1
225,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,10,4,NaN,Text,R20210209085637990407,1,1
226,5,2,AgeGenderSheet,Ages,Age Table,Age Table,2,2,Age,Ages,...,14.0,4.0,Gender Table,11,4,NaN,Text,R20210209085637990407,1,1


In [60]:
DROP TABLE IF EXISTS Working.PublishDataSet; 

select
--ds.[RowID]
ds.[ColumnFromReferenceText]
,ds.[DataSetID]
,ds.[FieldDescription]
,ds.[FieldID] as FieldName
,ds.[FieldPosition]
,ds.[ReferenceTextEnd]
,ds.[ReferenceTextStart]
,ds.[RowFromReferenceText]
,ds.[SheetName]
,ds.[TableOrientation]
--,dc.[SheetName]
,dc.[CellType]
,dc.[ColumnID]
,dc.[RowID]
,dc.[CellValue]
into Working.PublishDataSet from
working.DataSetTable ds, working.DataCellData dc
where
dc.SheetName = ds.SheetName and
dc.ColumnID = ds.ColumnFromReferenceText + @AnchorColumn and dc.RowID > ds.RowFromReferenceText + @AnchorRow;


--------------------- prepare for pivot -------------------------------------------------
DROP TABLE IF EXISTS Working.fieldnames; 

select FieldName into Working.fieldnames from Working.PublishDataSet;

set  @fieldnameslist = NULL

SELECT @fieldnameslist = ISNULL(@fieldnameslist + ',','') + QUOTENAME(FieldName)
FROM (SELECT DISTINCT FieldName FROM Working.fieldnames) AS FieldName;

DROP TABLE if exists Working.celldata; 
DROP TABLE if exists Working.pivoteddata;

--select 'DataSetDataSet';

--select * from Working.DataSetDataSet;


select * into Working.celldata from Working.PublishDataSet;

------- Do Pivot ----------

--Prepare the PIVOT query using the dynamic 
SET @DynamicPivotQuery = 
N'select * into Working.pivoteddata from (select RowID,FieldName,CellValue from Working.celldata) src
PIVOT(max(CellValue) for FieldName in ('+@fieldnameslist+')) piv;';
--Execute the Dynamic Pivot Query

--select @DynamicPivotQuery;

EXEC sp_executesql @DynamicPivotQuery;

---- Populate Pivoted Table

DROP TABLE IF EXISTS Working.PublishTable; 


SyntaxError: invalid syntax (<ipython-input-60-958f662ad936>, line 1)

In [ ]:
--------
set @createtable = NULL;

SELECT @createtable = ISNULL(@createtable + ',','') + CreateField
FROM (SELECT '['+ColumnName+']' + ' ' + ColumnType as CreateField FROM [Working].[DataMapTable]) AS CreateField;

SELECT @columnlist = ISNULL(@columnlist + ',','') + FieldName
FROM (SELECT '['+ColumnName+']' FieldName FROM [Working].[DataMapTable]) AS FieldName;

--SELECT ColumnName + ' ' + ColumnType as CreateField FROM [Working].[DataMapTable]

-- @TableSchema = [Schema]
-- @TableName = [TableName]

set @createtable = 'create table ' + @TableSchema +'.' + @TableName + '(' +@createtable + ')'

--select @createtable;

set @SQLCmnd = 'DROP TABLE IF EXISTS ' + @TableSchema +'.' + @TableName

EXEC sp_executesql @SQLCmnd;

EXEC sp_executesql @createtable;

set @SQLCmnd = 'insert into ' + @TableSchema +'.' + @TableName + ' select ' + @columnlist + ' from Working.pivoteddata;'

--select @SQLCmnd

EXEC sp_executesql @SQLCmnd;

DROP TABLE IF EXISTS Working.pivoteddata;

commit transaction;

--select * from Working.PublishTable;

GO